In [1]:
import pandas as pd
import numpy as np
from abtem.atoms import orthogonalize_cell
from ase.visualize import view
from ase.io import read, write
from ase.build import bulk, make_supercell
import py3Dmol
import sys
sys.path.append('../scripts/')
from mol_format import PDBFormatter, DataFrameConverter, XYZFormatter
from sklearn.neighbors import NearestNeighbors
import networkx as nx
from collections import Counter
from ase import Atoms

In [2]:
def create_supercell(structure, transformation_matrix):
    new_struc, strain = orthogonalize_cell(structure, return_transform=True)
    supercell = make_supercell(new_struc, transformation_matrix)
    return supercell

def visualize_structure(pdb_data, width=400, height=300):
    viewer = py3Dmol.view(width=width, height=height)
    viewer.addModel(pdb_data, 'pdb')
    viewer.setStyle({'stick': {}, 'sphere': {'radius': 0.3, 'colorscheme': 'Jmol'}})
    viewer.zoomTo()
    viewer.show()

def structure_to_dataframe(structure, res_name='SIO'):
    df = pd.DataFrame(structure.get_positions(), columns=['x', 'y', 'z'])
    df['atom_name'] = structure.get_chemical_symbols()
    df['res_name'] = res_name
    return df

def closest_neighbor_distances(array1, array2):
    # Initialize the NearestNeighbors model with 1 neighbor (closest)
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(array2)
    # Find the distance to the nearest neighbor for each point in array1
    distances, _ = nbrs.kneighbors(array1)
    # Flatten distances to return a 1D array
    return distances.flatten()

def neighbors_within_radius_withoutPBC(array1, array2, r_c):
    # Initialize NearestNeighbors with the radius parameter
    nbrs = NearestNeighbors(radius=r_c, algorithm='auto').fit(array2)
    # Find neighbors within radius for each point in array1
    indices = nbrs.radius_neighbors(array1, return_distance=False)
    return indices

def neighbors_within_radius(array1, array2, r_c, a, b, c):
    """
    Finds neighbors within a radius r_c, considering periodic boundary conditions.

    Parameters:
        array1 (np.ndarray): Array of query points (N x 3).
        array2 (np.ndarray): Array of points to search neighbors in (M x 3).
        r_c (float): Cutoff radius for neighbor search.
        a (float): Lattice parameter in x-direction.
        b (float): Lattice parameter in y-direction.
        c (float): Lattice parameter in z-direction.

    Returns:
        list: Indices of neighbors in the original array2 for each point in array1.
    """
    # Create periodic images of array2 within one layer of the unit cell
    translations = [
        np.array([dx * a, dy * b, dz * c])
        for dx in [-1, 0, 1]
        for dy in [-1, 0, 1]
        for dz in [-1, 0, 1]
    ]
    
    # Generate all periodic images of array2
    expanded_array2 = np.vstack([array2 + t for t in translations])

    # Map indices from expanded array back to the original array
    expanded_to_original = np.tile(np.arange(len(array2)), len(translations))

    # Use NearestNeighbors to find neighbors within the cutoff radius
    nbrs = NearestNeighbors(radius=r_c, algorithm='auto').fit(expanded_array2)
    expanded_indices = nbrs.radius_neighbors(array1, return_distance=False)

    # Convert indices back to the original array
    original_indices = [np.unique(expanded_to_original[indices]) for indices in expanded_indices]

    return original_indices

def create_graph_with_neighbors(df, neighbors, array1_label='Si', array2_label='O'):
    # Initialize the graph
    G = nx.Graph()
    
    # Add nodes and edges based on neighbors
    for i, neighbor_indices in enumerate(neighbors):
        # Create a node label for the Si atom
        si_node = f"{array1_label}_{i}"
        G.add_node(si_node)
        
        # Add nodes and edges for each neighbor
        for neighbor_idx in neighbor_indices:
            o_node = f"{array2_label}_{neighbor_idx}"
            G.add_node(o_node)
            G.add_edge(si_node, o_node)

    nodes = list(set(df.node_name.unique()) - set(G.nodes))
    for node in nodes:
        G.add_node(node)

    return G

def make_graph(df, struc, r_si_c=1.68):
    a, b, c = np.diag(struc.cell)
    neighbors = neighbors_within_radius(df.loc[df.atom_name == "Si", list('xyz')].values, 
                                        df.loc[df.atom_name == "O", list('xyz')].values, 
                                        r_si_c, a, b, c)
    
    df.loc[df.atom_name == "Si", 'node_name'] = \
        [f'Si_{i}' for i in range(df.query('atom_name == "Si"').shape[0])]
    df.loc[df.atom_name=="O", 'node_name'] =\
        [f'O_{i}' for i in range(df.query('atom_name == "O"').shape[0])]

    G_slab = create_graph_with_neighbors(df, neighbors)

    return G_slab

def dataframe_to_structure(df, original_structure):
    symbols = df['atom_name'].tolist()
    positions = df[['x', 'y', 'z']].values
    cell = original_structure.get_cell()
    pbc = original_structure.get_pbc()
    structure = Atoms(symbols=symbols, positions=positions, cell=cell, pbc=pbc)
    return structure

# Load the SiO2 CIF file
sio2 = '../files/initial_structures/MFI.cif'
struc = read(sio2)


transformation_matrix = [[2, 0, 0], 
                         [0, 2, 0], 
                         [0, 0, 3]]
supercell = create_supercell(struc, transformation_matrix)
df = structure_to_dataframe(supercell)
# df = structure_to_dataframe(struc)
converter = DataFrameConverter(PDBFormatter)

pdb_data = converter.convert(df)
visualize_structure(pdb_data)

c:\Users\lt0828\AppData\Local\anaconda3\Lib\site-packages\ase\io\cif.py:401: UserWarning: crystal system 'orthorhombic' is not interpreted for space group Spacegroup(62, setting=1). This may result in wrong setting!
  warnings.warn(


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
def give_si_neib(target_node, G):
    neibors = set()
    for i in [set(G[node]) for node in G[target_node]]:
        neibors = neibors.union(i)
    neibors = neibors - {target_node}
    return neibors

def choose_al(G, N):
    si_nodes = [node for node in G.nodes if 'Si' in node]
    np.random.shuffle(si_nodes)
    al_nodes = set()
    n = 0
    while True:
        if n > 10000:
            print
            break        
        for node in si_nodes:
            n += 1
            neibors = give_si_neib(node, G)
            if neibors & al_nodes:            
                continue
            else:
                al_nodes.add(node)
            if len(al_nodes) >= N:
                break
        else:
            continue
        break

    return al_nodes

def find_si_to_cut(df, r):
    coords = df.loc[:, list('xyz')].values
    coords -= coords.mean(axis=0)
    nodes2cut = df.loc[(coords[:,:2] ** 2).sum(axis=1) ** 0.5 < r, 'node_name'].to_list()
    nodes2cut = list(filter(lambda x: x[0] == 'S', nodes2cut))
    return nodes2cut

def remove_not_connected_nodes(G):
    for node in list(G.nodes):
        if not G.degree(node):
            G.remove_node(node)

def find_cutted_neib(G, si2cut):
    si2cut_neighbors = []
    for i in si2cut:
        for j in G.neighbors(i):
            si2cut_neighbors.append(j)
    return si2cut_neighbors

def add_h_atoms(dft, open_oxygens):
    coords_original = dft.loc[dft.node_name.isin(open_oxygens), list('xyz')].values
    coords = coords_original - coords_original.mean(axis=0)
    coords = coords / ((coords ** 2).sum(axis=1) ** 0.5)[: , np.newaxis]
    h_coords = coords_original - coords * 1
    h = pd.DataFrame(h_coords, columns=['x', 'y', 'z'])
    h['atom_name'] = 'H'
    h['res_name'] = 'SIO'
    dft = pd.concat([dft, h], ignore_index=True)
    return dft

def find_movable(df, G, si2cut):
    si_neighbors = set(find_cutted_neib(G, si2cut))
    open_oxygens = si_neighbors & set(G.nodes)
    df.loc[df.node_name.isin(open_oxygens), 'is_movable'] = True
    df.loc[~df.node_name.isin(open_oxygens), 'is_movable'] = False
    return open_oxygens

def make_hole(df, r, supercell):
    G = make_graph(df, supercell, r_si_c=1.8)
    si2cut = find_si_to_cut(df, r)
    open_oxygens = find_movable(df, G, si2cut)    
    G.remove_nodes_from(si2cut)
    remove_not_connected_nodes(G)
    dft = df[df.node_name.isin(G.nodes)].copy()
    dft = add_h_atoms(dft, open_oxygens)
    dft['is_movable'] = dft['is_movable'].fillna(True)
    return dft, G

def sort_atoms(df):
    df = df.sort_values(by=['is_movable', 'atom_name'])
    return df


r = 10
ratio_si_al = 75

dft, G = make_hole(df, r, supercell)
n_si = dft.atom_name.value_counts()['Si']

n_al = n_si // (1 + ratio_si_al)

al_nodes = choose_al(G, n_al)
dft.loc[dft.node_name.isin(al_nodes), 'atom_name'] = 'Al'
converter = DataFrameConverter(PDBFormatter)
pdb_data = converter.convert(dft)
visualize_structure(pdb_data)

new_structure = dataframe_to_structure(dft, supercell)
write(f'../files/processed_files/MFI_hole_radius_{int(r)}_ratio_{ratio_si_al}.cif', new_structure)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol